## Imports

In [77]:
import pandas as pd
import numpy as np

import torch
from tqdm import tqdm
from torch import nn
from transformers import AutoTokenizer
from torch.utils.data import Dataset, DataLoader
from timeit import default_timer as timer
from os import walk
from pathlib import Path
from sklearn.model_selection import train_test_split

## Data

In [78]:
ROOT_DATA_RAW = './'
HUMAN_JSON_FILE_NAME = 'human.jsonl'
HUMAN_JSON_PATH = f'{ROOT_DATA_RAW}/{HUMAN_JSON_FILE_NAME}'
MODELS_JSON_FOLDER_PATH = f'{ROOT_DATA_RAW}/machines'

In [79]:
BATCH_SIZE = 32
LSTM_UNITS = 256
LSTM_LAYERS = 5
EMBEDDING_SIZE = 512

In [80]:
TEST_SET_FRACTION = 0.3

In [81]:
df = pd.read_json(path_or_buf=HUMAN_JSON_PATH, lines=True)
df['is_llm'] = 0
df['dataset_name'] = Path(HUMAN_JSON_FILE_NAME).stem

In [82]:
dir_path, dir_names, file_names = next(walk(MODELS_JSON_FOLDER_PATH))

print(dir_names)
for file_name in file_names:
    temp_df = pd.read_json(path_or_buf=f'{MODELS_JSON_FOLDER_PATH}/{file_name}', lines=True)
    temp_df['is_llm'] = 1
    temp_df['dataset_name'] = Path(file_name).stem

    df = pd.concat([df, temp_df], ignore_index=True)

df.drop(labels=['id'], inplace=True, axis='columns')

[]


In [83]:
df.tail()

,text,is_llm,dataset_name
15213,Gabby Petito Case: Social Media Detectives and...,1,text-bison-002
15214,Search Intensifies for Missing Woman Gabby Pet...,1,text-bison-002
15215,University of Wisconsin Oshkosh Student Claims...,1,text-bison-002
15216,Gabby Petito Case: The Internet's Role in a Tr...,1,text-bison-002
15217,Gabby Petito Remembered as a 'Super Kind-Heart...,1,text-bison-002


### Tokenize

In [84]:
checkpoint = 'distilbert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [85]:
df['tokenized_text'] = tokenizer(list(df['text'].to_list()))['input_ids']

Token indices sequence length is longer than the specified maximum sequence length for this model (843 > 512). Running this sequence through the model will result in indexing errors


## Model

In [86]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [87]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_size, layers_num, device, output_size=1, dropout=0):
        super().__init__()

        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        self.hidden_size = hidden_size
        self.layers_num = layers_num
        self.output_size= output_size
        self.dropout = dropout
        self.device = device

        self.embed = nn.Embedding(self.vocab_size, self.embedding_size, device=self.device)

        self.lstm = nn.LSTM(
            input_size=self.embedding_size,
            hidden_size=self.hidden_size,
            num_layers=self.layers_num,
            batch_first=True,
            dropout=self.dropout,
            device=self.device
        )

        self.fc = nn.Linear(
            self.hidden_size,
            self.output_size
        )

    def forward(self, X, lengths):
        embeddings = self.embed(X)

        seq_output, (h_n, c_n) = self.lstm(embeddings)

        out = seq_output.sum(dim=1).div(lengths.float().unsqueeze(dim=1))
        logits = self.fc(out)
        return logits

## Dataset

In [88]:
class TextDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return self.X.size

    def __getitem__(self, index):
        return (
            self.X[index],
            self.y[index]
        )

In [89]:
def collate_fn(batch):
  # We want to sort the batch by seq length,
  # in order to make the computation more efficient
  batch = sorted(batch, key=lambda x: len(x[0]), reverse=True)

  inputs = [torch.LongTensor(x[0]).to(device) for x in batch]
  padded_input = nn.utils.rnn.pad_sequence(inputs, batch_first=True)

  lengths = torch.LongTensor([len(x[0]) for x in batch]).to(device)

  y = torch.FloatTensor(np.array([x[1] for x in batch])).reshape(-1, 1).to(device)

  return padded_input, lengths, y

## Train and test functions

In [90]:
def calculate_accuracy(y_true, y_hat):
    correct_pred = torch.eq(torch.sigmoid(y_hat).round(), y_true).sum().item()
    return (correct_pred / len(y_hat)) * 100

def train_step(model, dataloader, loss_fn, optimizer, device):

    model.train()

    train_loss, train_acc = 0, 0
    steps = 0

    for X, lengths, y in dataloader:
        X, y = X.to(device), y.to(device)

        y_hat = model(X, lengths)

        loss = loss_fn(y_hat, y)
        train_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_acc += calculate_accuracy(y_true=y, y_hat=y_hat)
        steps += 1

    return train_loss / steps, train_acc / steps


def test_step(model, dataloader, loss_fn, device):

    model.eval()

    test_loss, test_acc = 0, 0
    steps = 0

    with torch.inference_mode():
        for X, lengths, y in dataloader:

            X, y = X.to(device), y.to(device)

            y_hat = model(X, lengths)

            loss = loss_fn(y_hat, y)
            test_loss += loss.item()

            test_acc += calculate_accuracy(y_true=y, y_hat=y_hat)
            steps += 1

    return test_loss / steps, test_acc / steps

def train(model,
          train_dataloader,
          test_dataloader,
          optimizer,
          loss_fn,
          epochs,
          device):

    results = {
        "train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": []
        }

    model.to(device)

    for epoch in tqdm(range(epochs)):

        start_time = timer()
        train_loss, train_acc = train_step(
            model=model,
            dataloader=train_dataloader,
            loss_fn=loss_fn,
            optimizer=optimizer,
            device=device,
        )
        end_time = timer()

        test_loss, test_acc = test_step(
            model=model,
            dataloader=test_dataloader,
            loss_fn=loss_fn,
            device=device,
        )

        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"test_acc: {test_acc:.4f} | "
            f"time: {(end_time-start_time):.4f}"
        )

    return results

## Train on each LLM dataset separately and test against all others

In [91]:
human_df = df.loc[df['is_llm'] == 0]
human_train_df, human_test_df = train_test_split(human_df, test_size=TEST_SET_FRACTION)

In [92]:
for llm_name in df['dataset_name'].unique():
    if llm_name == Path(HUMAN_JSON_FILE_NAME).stem:
        continue

    llm_df = df.loc[df['dataset_name'] == llm_name]
    llm_train_df, llm_test_df = train_test_split(llm_df, test_size=TEST_SET_FRACTION)

    train_df = pd.concat([human_train_df, llm_train_df], ignore_index=True)
    test_df = pd.concat([human_test_df, llm_test_df], ignore_index=True)

    train_dataset = TextDataset(train_df['tokenized_text'], train_df['is_llm'])
    test_dataset = TextDataset(test_df['tokenized_text'], test_df['is_llm'])

    train_dataloader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        drop_last=False,
        collate_fn=collate_fn
    )

    test_dataloader = DataLoader(
        test_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        drop_last=False,
        collate_fn=collate_fn
    )

    model = RNN(tokenizer.vocab_size, EMBEDDING_SIZE, LSTM_UNITS, LSTM_LAYERS, device, dropout=0.6).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.00008)
    loss_fn = nn.BCEWithLogitsLoss()

    print(f'Training model for {llm_name}...')

    current_results = train(
        model,
        train_dataloader,
        test_dataloader,
        optimizer,
        loss_fn,
        epochs=15,
        device=device
    )

    print(f'Finished training model for {llm_name}')

Training model for gemini-pro...


  7%|▋         | 1/15 [00:16<03:55, 16.83s/it]

Epoch: 1 | train_loss: 0.6938 | train_acc: 48.4375 | test_loss: 0.6914 | test_acc: 57.7806 | time: 14.5844


 13%|█▎        | 2/15 [00:34<03:41, 17.07s/it]

Epoch: 2 | train_loss: 0.6932 | train_acc: 47.3307 | test_loss: 0.6930 | test_acc: 51.3393 | time: 14.9668


...

Epoch: 13 | train_loss: 0.6971 | train_acc: 49.3490 | test_loss: 0.7010 | test_acc: 51.3393 | time: 14.9905


 93%|█████████▎| 14/15 [04:01<00:17, 17.22s/it]

Epoch: 14 | train_loss: 0.6935 | train_acc: 48.8281 | test_loss: 0.6945 | test_acc: 42.4107 | time: 14.8754


100%|██████████| 15/15 [04:18<00:00, 17.25s/it]

Epoch: 15 | train_loss: 0.6955 | train_acc: 50.1953 | test_loss: 0.7043 | test_acc: 51.3393 | time: 14.9792
Finished training model for text-bison-002
